In [1]:
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) 
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt  # plot 
from sklearn.model_selection import train_test_split, KFold, cross_val_score #  kflod validation
from keras.models import Sequential # model 
from keras.models import load_model
from keras.layers import Dense, Dropout  # dropout
from sklearn.model_selection import train_test_split # 划分数据集

Using TensorFlow backend.


In [2]:
#　查看是否在使用gpu进行训练
print(tf.test.is_gpu_available())

False


In [3]:
#　查看使用的是哪一个虚拟环境
!which python

/home/mastlab-zl/anaconda3/envs/tf-cpu/bin/python


In [4]:
# 定义动作类型
action = {
    0: 'Stand',
    1: 'Wave',
    2: 'Flap',
    3: 'Squat',
    4: 'Bowling',
}

In [5]:
# 准备数据
df = pd.read_csv('output_normalized_cmu.csv', header=0)
seed = 42

In [6]:
# x为36维特征
X = df.loc[:, :'17_y'].astype(float)
# y为动作种类（标签）
Y = df.loc[:,['type_index'] ]
encoder =  LabelBinarizer()
dummy_Y = encoder.fit_transform(Y)

In [7]:
X.values

array([[0.5 , 0.02, 0.5 , ..., 0.03, 0.75, 0.03],
       [0.6 , 0.02, 0.53, ..., 0.03, 0.73, 0.05],
       [0.57, 0.02, 0.57, ..., 0.02, 0.79, 0.02],
       ...,
       [0.42, 0.06, 0.47, ..., 0.03, 0.58, 0.  ],
       [0.71, 0.04, 0.5 , ..., 0.02, 0.86, 0.06],
       [0.94, 0.21, 0.59, ..., 0.11, 0.  , 0.  ]])

In [8]:
Y

,type_index
0,0
1,0
2,0
3,0
4,0
...,...
2190,4
2191,4
2192,4
2193,4


In [9]:
# 导入训练好的模型
model = load_model('dnn_model.h5')





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [10]:
start_time = time.time()
# 找出所有识别错误的图片
wrong_dict = {}
for i in range(len(X)):
    input_x = X.values[i, :].reshape((1,36))
    right_y = int(Y.loc[i])
    pred_y = model.predict(input_x)
    pred_y=encoder.inverse_transform(pred_y)[0]
    if pred_y != right_y:
        wrong_dict[str(df.iloc[i, 38])]= pred_y

elapsed_time = time.time() - start_time

In [11]:
print("以下图片识别错误：")
for key,value in wrong_dict.items():
    print('图片',key,'被识别为:', action[value])
print('%d张图片识别完毕, 耗时%.2f秒, 共有%d张图片识别错误' % (len(X), elapsed_time, len(wrong_dict)))

以下图片识别错误：
图片 ./input/stand/Colour 40-misc-145.png 被识别为: Wave
图片 ./input/stand/Colour 45-misc-145.png 被识别为: Wave
图片 ./input/flap/Colour 180-146-flap.png 被识别为: Wave
图片 ./input/squat/Colour 777-160-squat.png 被识别为: Stand
图片 ./input/squat/Colour 778-160-squat.png 被识别为: Bowling
图片 ./input/squat/Colour 779-160-squat.png 被识别为: Wave
图片 ./input/squat/Colour 829-99-squat.png 被识别为: Stand
图片 ./input/squat/Colour 827-99-squat.png 被识别为: Stand
图片 ./input/squat/Colour 826-99-squat.png 被识别为: Stand
图片 ./input/bowling/Colour 223 (29)-bowling.png 被识别为: Squat
图片 ./input/bowling/Colour 136 (28)-bowling.png 被识别为: Flap
图片 ./input/bowling/Colour 136 (25)-bowling.png 被识别为: Flap
图片 ./input/bowling/Colour 136 (20)-bowling.png 被识别为: Flap
图片 ./input/bowling/Colour 73 (11)-bowling.png 被识别为: Stand
2195张图片识别完毕, 耗时1.87秒, 共有14张图片识别错误
